# 10_05 10_05 Naive Bayes Classifier (10-05 나이브 베이즈 분류기)


출처: https://wikidocs.net/22892




## (1) 뉴스그룹 데이터에 대한 이해

사이킷 런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 18,846개의 뉴스그룹 데이터를 제공 (토픽 모델링의 LSA 챕터에서 사용했던 데이터와 동일한 데이터)

훈련데이터와 테스트 데이터를 미리 구별. 

subset 부분에 'all'을 넣으면 18,846개의 전체 데이터 다운로드할 수 있으며, 'train'을 넣으면 훈련 데이터를, 'test'를 넣으면 테스트 데이터를 다운로드할 수 있음. 

newsdata.keys()를 출력하여 해당 데이터가 어떤 속성으로 구성되어져 있는지 출력

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


훈련용 샘플의 개수를 확인해보겠습니다.



In [4]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


어떤 카테고리들로 구성되어있는지 확인

In [5]:
print(newsdata.target_names)


['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


target에는 총 0부터 19까지의 숫자가 들어가있는데 첫번째 샘플의 경우에는 몇 번 카테고리인지 확인

In [6]:
print(newsdata.target[0])


7


In [7]:
print(newsdata.target_names[7])


rec.autos


7번 카테고리의 카테고리 제목은 rec.autos입니다. 즉, 첫번째 샘플은 rec.autos 카테고리에 속합니다. 그렇다면 첫번째 샘플이 어떤 내용을 갖고 있는지 확인

In [8]:
print(newsdata.data[0])


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







스포츠 카에 대한 글

총 0번부터 19까지의 카테고리 중 7번 레이블에 속하는 글,
7번은 rec.autos 카테고리.

## (2) 나이브 베이즈 분류

운로드 받은 훈련 데이터에 대한 전처리를 진행

사용할 데이터는 newsdata.data
그에 대한 카테고리 레이블이 되어있는 newsdata.target

전처리를 해야하는 데이터는 newsdata.data- > 토큰화를 해야함. 
. 나이브 베이즈 분류를 위해서는 데이터를 BoW로 만들어줄 필요

### 입력텍스트를 자동으로 BoW로 만드는 CountVectorizer를 사용 (BoW 챕터 및 DTM 챕터 참고)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [10]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


이제 자동으로 DTM이 완성

11,314는 훈련용 샘플의 개수 - DTM관점에서는 문서의 수

130,107은 전체 훈련 데이터에 등장한 단어의 수를 의미
 
### DTM 그대로 사용 가능 -> 이번에는 TF-IDF가중치를 적용한 행렬을 입력으로 텍스트 분류를 수행하면, 성능의 개선이 가능. 

*  TF-IDF 행렬이 항상 DTM으로 수행했을 때보다 성능이 뛰어나지는 않습니다.

* 사이킷런은 TF-IDF를 자동 계산해주는 TfidVectorizer 클래스를 제공하므로 이를 사용

In [12]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


이제 TF-IDF 행렬이 만들어졌습니다. 이제 본격적으로 나이브 베이즈 분류를 수행해보겠습니다. 사이킷런은 나이브 베이즈 모델을 지원하므로, 이를 그대로 갖고와서 사용

모델의 입력으로 TF-IDF 행렬과 11,314개의 훈련 데이터에 대한 레이블이 적혀있는 newsdata.target이 들어갑니다. 이는 앞서 배운 분류 예제들을 상기해보면, 각각 X_train과 y_train에 해당되는 데이터

In [13]:
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB()

여기서 alpha=1.0은 라플라스 스무딩이 적용되었음을 의미합니다.

* 라플라스 스무딩: 그런데 예제를 보니 이상한 점이 보입니다. 물론, 직관적으로 보기에도 you, free, lottery라는 단어가 스팸 메일에서 빈도수가 더 높기때문에 스팸 메일인 확률이 더 높은 것은 확실합니다. 

* 그러나 입력 텍스트에 대해서 단, 하나의 단어라도 훈련 텍스트에 없었다면 확률 전체가 0이 되는 것은 지나친 일반화입니다. 이 경우에는 정상 메일에 lottery가 단 한 번도 등장하지 않았고, 그 이유로 정상 메일일 확률 자체가 0%가 되어버렸습니다.

* 이를 방지하기 위해서 나이브 베이즈 분류기에서는 각 단어에 대한 확률의 분모, 분자에 전부 숫자를 더해서 분자가 0이 되는 것을 방지하는 라플라스 스무딩을 사용하기도 합니다.


In [14]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


MultinomialNB()

In [15]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514


77%의 정확도를 얻었습니다. 여기서는 하지 않았지만, 잠재 의미 분석 챕터에서 진행했던 전처리를 모두 진행하고 다시 나이브 베이즈 분류기를 돌려보세요. 80% 이상의 정확도를 얻을 수 있습니다.